In [114]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [115]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [116]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [117]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction


In [118]:
servicio = pd.read_sql_table("mensajeria_estadosservicio",url_db)

In [119]:
total_nan = servicio.isna().sum()
print(total_nan)
print(servicio.columns)

id                    0
fecha                 0
hora                  0
foto                  0
observaciones         1
estado_id             0
servicio_id           0
es_prueba             0
foto_binary      128132
dtype: int64
Index(['id', 'fecha', 'hora', 'foto', 'observaciones', 'estado_id',
       'servicio_id', 'es_prueba', 'foto_binary'],
      dtype='object')


In [120]:
print(len(servicio))

128402


## Transformation

In [121]:

servicio["fase"] = servicio["estado_id"]
# Hacer el merge asegurando que coincidan
hecho_servicio_fase = servicio[["id", "fase","fecha","hora","servicio_id"]]
hecho_servicio_fase.head(5)

,id,fase,fecha,hora,servicio_id
0,1014,4,2024-01-29,01:13:32,226
1,1484,3,2024-01-30,18:45:12,79
2,2829,5,2024-02-06,11:34:04,613
3,1888,4,2024-02-01,14:50:39,376
4,32312,3,2024-04-06,16:11:21,7164


## Calculo atributo tiempo promedio general

In [122]:


# Asegúrate de que 'fecha' esté en formato datetime
hecho_servicio_fase.loc[:, 'fecha'] = pd.to_datetime(hecho_servicio_fase['fecha'])
print(hecho_servicio_fase["fecha"].dtype)

# Convierte la columna 'hora' a cadena
hecho_servicio_fase.loc[:, "hora"] = hecho_servicio_fase["hora"].astype(str)

# Asegúrate de que el formato de la hora sea adecuado
# Ya tienes la hora en el formato correcto, puedes imprimirla para verificar
print(hecho_servicio_fase["hora"].head())

# Combina la fecha y hora en un solo datetime
hecho_servicio_fase['fecha_hora'] = pd.to_datetime(hecho_servicio_fase['fecha'].dt.strftime('%Y-%m-%d') + ' ' + hecho_servicio_fase['hora'])

# Mostrar el resultado
print(hecho_servicio_fase[['fecha', 'hora', 'fecha_hora']].head())


datetime64[ns]
0    01:13:32
1    18:45:12
2    11:34:04
3    14:50:39
4    16:11:21
Name: hora, dtype: object


ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M:%S": ".603000", at position 676. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.